In [2]:
import scanpy as sc
import matplotlib as mpl
import cell2location
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os
import pandas as pd
import numpy as np
rcParams['pdf.fonttype'] = 42

/home/bailab/miniconda3/envs/cell2location/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
results_folder = '/data/project/AI4Omic/MASLD/results/ST/cell2location'
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'

In [4]:
sc.settings.figdir = results_folder
sc.settings.set_figure_params(frameon=False, dpi=300, dpi_save=300, fontsize=10, format='pdf')

In [ ]:
# Reload the c2l reulsts
adata_vis = sc.read_h5ad('/data/project/AI4Omic/MASLD/results/ST/cell2location/cell2location_map/adata_c2l.h5ad')

In [ ]:
sp_data_folder = '/data/project/AI4Omic/MASLD/data/Rawdata/Visium'
sample_names = [i for i in os.listdir(sp_data_folder)]
sample_names

In [5]:
def read_and_qc(sample_name,path,group_dict,force_filter=True):
    adata = sc.read_visium(path + str(sample_name) ,count_file='filtered_feature_bc_matrix.h5', load_images=True, library_id=sample_name)
    adata.obs['sample'] = sample_name
    adata.var['SYMBOL'] = adata.var_names
    adata.obs['group'] = group_dict.get(sample_name)
    #adata.var.set_index('gene_ids',drop=True,inplace=True)
    adata.var_names_make_unique()
    
    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.var['rps'] = [gene.startswith('RPS') for gene in adata.var['SYMBOL']]
    adata.var['mrp'] = [gene.startswith('MRP') for gene in adata.var['SYMBOL']]
    adata.var['rpl'] = [gene.startswith('RPL') for gene in adata.var['SYMBOL']]
    sc.pp.calculate_qc_metrics(adata,qc_vars=['mt'], inplace=True, percent_top=None, log1p=False)
    
    # add sample name to obs names
    adata.obs['sample'] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs['sample'] + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    adata.var['duplicated'] = adata.var['SYMBOL'].duplicated(keep = 'first')
    adata = adata[:, ~adata.var['duplicated'].values]

    #replacing adata.uns['spatial'] key name
    file = list(adata.uns['spatial'].keys())[0]
    adata.uns['spatial'][sample_name] = adata.uns['spatial'][file].copy()
    if file != sample_name:
        del adata.uns['spatial'][file]
    print(adata.uns['spatial'].keys())
    
    # Add group information
    adata.obs['group'] = group_dict.get(list(adata.uns['spatial'].keys())[0])

    # Save raw data
    adata.raw = adata.copy()

    if force_filter:
        #First filter:
        #spots with no information (less than 200 genes and 400 UMIs)
        sc.pp.calculate_qc_metrics(adata, inplace=True)
        #adata = adata[(adata.obs['n_genes_by_counts'].values > 200) & (adata.obs['total_counts'].values > 400),:]
        sc.pp.filter_cells(adata, min_genes=200)
        sc.pp.filter_cells(adata, min_counts=400)
        
        #Second filter: mt and rb genes
        #mitochondria-encoded (MT) genes should be removed for spatial mapping
        adata.obsm['mt'] = adata[:, adata.var['mt'].values|adata.var['rps'].values|adata.var['mrp'].values|adata.var['rpl'].values].X.toarray()
        adata = adata[:, ~(adata.var['mt'].values|adata.var['rps'].values|adata.var['mrp'].values|adata.var['rpl'].values)]

        #Third filter:
        #Genes expressed in less than 2 spots
        #adata = adata[:, adata.var['n_cells_by_counts'].values >= 3]
        sc.pp.filter_genes(adata, min_cells=3)
    return adata

In [ ]:
#A loop for read and qc ST data
slides = []
for i in sample_names:
    adata = read_and_qc(sample_name = i , path = sp_data_folder, group_dict = group_dict, force_filter= True)
    slides.append(adata)
    del adata

#Combine individual samples
adata_vis = slides[0].concatenate(slides[1:len(sample_names)],batch_key='sample',uns_merge='unique',batch_categories=sample_names,index_unique=None)
del slides

In [ ]:
# Load Visium ST data from STAMP results
adata_vis = sc.read_h5ad("/data/project/AI4Omic/MASLD/results/ST/STAMP/adata_concat.h5ad")
adata_vis.var['mt'] = [gene.startswith('MT-') for gene in adata_vis.var.index]
adata_vis.var['rps'] = [gene.startswith('RPS') for gene in adata_vis.var.index]
adata_vis.var['mrp'] = [gene.startswith('MRP') for gene in adata_vis.var.index]
adata_vis.var['rpl'] = [gene.startswith('RPL') for gene in adata_vis.var.index]
# mitochondria-encoded (MT) genes should be removed for spatial mapping
adata_vis.obsm['mt'] = adata_vis[:, adata_vis.var['mt'].values|adata_vis.var['rps'].values|adata_vis.var['mrp'].values|adata_vis.var['rpl'].values].X.toarray()
adata_vis = adata_vis[:, ~(adata_vis.var['mt'].values|adata_vis.var['rps'].values|adata_vis.var['mrp'].values|adata_vis.var['rpl'].values)]
adata_vis

View of AnnData object with n_obs × n_vars = 48154 × 22276
    obs: 'in_tissue', 'array_row', 'array_col', 'disease_status', 'sample', 'Age', 'Sex', 'Steatosis', 'Ballooning', 'Lobular_inflammation', 'Fibrosis', 'NAS', 'n_counts', 'n_genes', 'library_id', 'Topic1_score', 'Topic1', 'Topic2_score', 'Topic2', 'Topic3_score', 'Topic3', 'Topic4_score', 'Topic4', 'Topic5_score', 'Topic5', 'Topic6_score', 'Topic6', 'Topic7_score', 'Topic7', 'Topic8_score', 'Topic8', 'Topic9_score', 'Topic9', 'Topic10_score', 'Topic10', 'Topic11_score', 'Topic11', 'Topic12_score', 'Topic12', 'Topic13_score', 'Topic13', 'Topic14_score', 'Topic14', 'Topic15_score', 'Topic15', 'Topic16_score', 'Topic16', 'Topic17_score', 'Topic17', 'Topic18_score', 'Topic18', 'Topic19_score', 'Topic19', 'Topic20_score', 'Topic20'
    var: 'mt', 'rps', 'mrp', 'rpl'
    uns: 'disease_status_colors', 'spatial'
    obsm: 'spatial', 'mt'

In [29]:
# Find shared genes and prepare anndata. Subset both anndata and reference signatures
inf_aver = pd.read_csv(f'{ref_run_name}/inf_aver.csv', index_col=0)
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
adata_vis = adata_vis[:,intersect].copy()
inf_aver = inf_aver.loc[intersect,:].copy()
adata_vis

AnnData object with n_obs × n_vars = 48154 × 20347
    obs: 'in_tissue', 'array_row', 'array_col', 'disease_status', 'sample', 'Age', 'Sex', 'Steatosis', 'Ballooning', 'Lobular_inflammation', 'Fibrosis', 'NAS', 'n_counts', 'n_genes', 'library_id', 'Topic1_score', 'Topic1_x', 'Topic2_score', 'Topic2_x', 'Topic3_score', 'Topic3_x', 'Topic4_score', 'Topic4_x', 'Topic5_score', 'Topic5_x', 'Topic6_score', 'Topic6_x', 'Topic7_score', 'Topic7_x', 'Topic8_score', 'Topic8_x', 'Topic9_score', 'Topic9_x', 'Topic10_score', 'Topic10_x', 'Topic11_score', 'Topic11_x', 'Topic12_score', 'Topic12_x', 'Topic13_score', 'Topic13_x', 'Topic14_score', 'Topic14_x', 'Topic15_score', 'Topic15_x', 'Topic16_score', 'Topic16_x', 'Topic17_score', 'Topic17_x', 'Topic18_score', 'Topic18_x', 'Topic19_score', 'Topic19_x', 'Topic20_score', 'Topic20_x', '_indices', '_scvi_batch', '_scvi_labels', 'Bmemory', 'Bnaive', 'CD14+ Monocyte', 'CD16+ Monocyte', 'Central Vein EC', 'Cholangiocyte', 'Circulating NK cell', 'Circulatin

In [15]:
#prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key='sample')

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [16]:
#create and train the model
mod = cell2location.models.Cell2location(adata_vis, cell_state_df=inf_aver, N_cells_per_location=8, detection_alpha=20)
mod.view_anndata_setup()

Anndata setup with scvi-tools version 1.1.6.

Setup via `Cell2location.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': 'sample',
│   'labels_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │  35   │
│         n_cells          │ 48154 │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │   1   │
│          n_vars          │ 20347 │
└──────────────────────────┴───────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    ind_x     │   adata.obs['_indices']   │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                   batch State Registry                   
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location   ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['sample'] │  CTRL-161  │          0          │
│                     │  CTRL-179  │          1          │
│                     │  CTRL-180  │          2          │
│                     │ CTRL-5113  │          3          │
│                     │ CTRL-5759  │          4          │
│                     │ CTRL-8395  │          5          │
│                     │ CTRL-8715  │          6          │
│                     │ MASH-0413  │          7          │
│                     │ MASH-0422  │          8          │
│                     │ MASH-0835  │          9          │
│                     │ MASH-1086  │         10          │
│                     │ MASH-1475  │         11          │
│                     │ MASH-1478  │         12          │
│                     │ MASH-1480  │         13          │
│                     │ MASH-1481  │         14          │
│                     │ MASH-1494  │         15          │
│                     │ MASH-1501  │         16          │
│                     │ MASH-2534  │         17          │
│                     │ MASH-3096  │         18          │
│                     │ MASH-3344  │         19          │
│                     │ MASH-4426  │         20          │
│                     │ MASH-7866  │         21          │
│                     │ MASH-8684  │         22          │
│                     │ MASH-9136  │         23          │
│                     │ MASH-9440  │         24          │
│                     │ MASLD-0966 │         25          │
│                     │ MASLD-1479 │         26          │
│                     │ MASLD-1492 │         27          │
│                     │ MASLD-1493 │         28          │
│                     │ MASLD-1495 │         29          │
│                     │ MASLD-1497 │         30          │
│                     │ MASLD-1498 │         31          │
│                     │ MASLD-2768 │         32          │
│                     │ MASLD-4973 │         33          │
│                     │ MASLD-9993 │         34          │
└─────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

In [ ]:
#Training cell2location
mod.train(max_epochs=15000, batch_size=5000, train_size=1)

#plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(100)
plt.legend(labels=['full data training'])

In [26]:
#Exporting estimated posterior distributions of cell abundance and saving results
#adata_vis = mod.export_posterior(adata_vis, sample_kwargs={'num_samples':1000, 'batch_size':mod.adata.n_obs})
adata_vis = mod.export_posterior(adata_vis, sample_kwargs={'num_samples':1000, 'batch_size':5000})

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


Sampling global variables, sample: 100%|██████████| 999/999 [00:23<00:00, 42.81it/s]


In [27]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']

In [ ]:
mod.plot_QC()

In [ ]:
#Save and load model
mod.save(f"{run_name}", overwrite=True)
adata_vis = sc.read_h5ad('/data/project/AI4Omic/MASLD/results/ST/cell2location/cell2location_map/adata_c2l.h5ad')
#mod = cell2location.models.Cell2location.load(f'{run_name}',adata_vis)

#save anndata object with results
adata_file = f"{run_name}/adata_c2l.h5ad"
adata_vis.write(adata_file)
adata_file

# save .obs data
adata_vis.obs.to_csv(f"{run_name}/adata_c2l.obs.csv")

Spatial correlation between PPAR and MITF

In [ ]:
#PPAR Pathway scoring
PPAR_signaling = ["ACAA1", "ACADL", "ACADM", "ACOX1", "ACOX2", "ACOX3", "ACSBG1", "ACSBG2","ACSL1", "ACSL3", "ACSL4", "ACSL5", "ACSL6", "ADIPOQ", "ANGPTL4", "APOA1","APOA2", "APOA5", "APOC3", "AQP7", "CD36", "CPT1A", "CPT1B", "CPT1C","CPT2", "CYP27A1", "CYP4A11", "CYP7A1", "CYP8B1", "DBI", "EHHADH", "FABP1","FABP2", "FABP3", "FABP4", "FABP5", "FABP6", "FABP7", "FADS2", "GK2","GK3", "HMGCS2", "ILK", "LPL", "ME1", "MMP1", "NR1H3", "OLR1","PCK1", "PCK2", "PDPK1", "PLIN1", "PLTP", "PPARA", "PPARD", "PPARG","RXRA", "RXRB", "RXRG", "SCD", "SCP2", "SLC27A1", "SLC27A2", "SLC27A4","SLC27A5", "SLC27A6", "SORBS1", "UCP1"]
sc.tl.score_genes(adata_vis, gene_list=PPAR_signaling, score_name='PPAR')

#MITF Regulon scoring
adjacencies = pd.read_csv('/data/project/AI4Omic/MASLD/results/scRNA/pyscenic/adj.tsv', index_col=False, sep='\t')
MITF_targets = adjacencies.loc[adjacencies.TF == 'MITF',:].sort_values('importance', ascending=False).target[0:50]
sc.tl.score_genes(adata_vis, gene_list=MITF_targets, score_name='MITF')
#adjacencies.loc[adjacencies.target == 'SDC1',:]

In [ ]:
slide = "MASH-1494"
sc.pl.spatial(adata_vis[adata_vis.obs['sample']==slide], color=['MITF', 'PPAR'],library_id=slide, cmap='RdBu_r',size=1.5, img_key=None, wspace=0.15, frameon=True, colorbar_loc=None, save=slide+'_MITF_PPAR', add_outline=True)
slide = "MASH-1501"
sc.pl.spatial(adata_vis[adata_vis.obs['sample']==slide], color=['MITF', 'PPAR'],library_id=slide, cmap='RdBu_r',size=1.5, img_key=None, wspace=0.15, frameon=True, colorbar_loc=None, save=slide+'_MITF_PPAR', add_outline=True)
slide = "MASH-1481"
sc.pl.spatial(adata_vis[adata_vis.obs['sample']==slide], color=['MITF', 'PPAR'],library_id=slide, cmap='RdBu_r',size=1.5, img_key=None, wspace=0.15, frameon=True, colorbar_loc=None, save=slide+'_MITF_PPAR', add_outline=True)

In [ ]:
# select one slide
from cell2location.utils import select_slide
slide = select_slide(adata_vis, 'MASH-1494')

# plot in spatial coordinates
with mpl.rc_context({'axes.facecolor':  'black','figure.figsize': [4.5, 5]}):
    sc.pl.spatial(slide, cmap='magma',
                  # show first 8 cell types
                  color=['HSCs', 'Cholangiocytes', 'LAMs', 'Plasma cells', 'T&NK', 'KCs&moKCs', 'LECs', 'cDC1s', 'cDC2s', 'pDCs', 'Neutrophils', 'Mast cells', 'Monocytes', 'B-cells'],
                  ncols=4, size=1.3,
                  img_key='hires',
                  # limit color scale at 99.2% quantile of cell abundance
                  vmin=0, vmax='p99.2'
                 )

In [20]:
help(plot_spatial)

Help on function plot_spatial in module cell2location.plt.plot_spatial:

plot_spatial(adata, color, img_key='hires', show_img=True, **kwargs)
    Plot spatial abundance of cell types (regulatory programmes) with colour gradient
    and interpolation (from Visium anndata).

    This method supports only 7 cell types with these colours (in order, which can be changed using reorder_cmap).
    'yellow' 'orange' 'blue' 'green' 'purple' 'grey' 'white'

    :param adata: adata object with spatial coordinates in adata.obsm['spatial']
    :param color: list of adata.obs column names to be plotted
    :param kwargs: arguments to plot_spatial_general
    :return: matplotlib figure



In [ ]:
# Now we use cell2location plotter that allows showing multiple cell types in one panel
from cell2location.plt import plot_spatial
from cell2location.utils import select_slide
# select up to 6 clusters
clust_labels = ['Central Vein EC', 'HSC']
clust_col = ['' + str(i) for i in clust_labels] # in case column names differ from labels
sample = 'MASH-3096'
slide = select_slide(adata_vis, sample)

with mpl.rc_context({'figure.figsize': (8, 8.5), 'axes.grid': False}):
    fig = plot_spatial(
        adata = slide,
        # labels to show on a plot
        color = clust_col, labels = clust_labels,
        show_img = True,
        # 'fast' (white background) or 'dark_background'
        style = 'fast',
        # limit color scale at 99.2% quantile of cell abundance
        max_color_quantile = 0.992,
        # size of locations (adjust depending on figure size)
        circle_diameter = 3.5,
        colorbar_position = 'bottom',
        reorder_cmap = [0, 4, 2, 1, 3, 5, 6],
        #crop_x = [620, 750],
        #crop_y = [800, 930],
        colorbar_shape = {'vertical_gaps': 0.15}
    )
    fig.savefig(f'/data/project/AI4Omic/MASLD/results/ST/cell2location/{sample}_{clust_col}'+'.pdf', dpi=300, format='pdf', bbox_inches='tight')

In [ ]:
# Now we use cell2location plotter that allows showing multiple cell types in one panel
from cell2location.plt import plot_spatial
from cell2location.utils import select_slide
# select up to 6 clusters
clust_labels = ['Central Vein EC', 'HSC']
clust_col = ['' + str(i) for i in clust_labels] # in case column names differ from labels
sample = 'MASH-3096'
slide = select_slide(adata_vis, sample)

with mpl.rc_context({'figure.figsize': (8, 8.5), 'axes.grid': False}):
    fig = plot_spatial(
        adata = slide,
        # labels to show on a plot
        color = clust_col, labels = clust_labels,
        show_img = True,
        # 'fast' (white background) or 'dark_background'
        style = 'fast',
        # limit color scale at 99.2% quantile of cell abundance
        max_color_quantile = 0.992,
        # size of locations (adjust depending on figure size)
        circle_diameter = 3.5,
        colorbar_position = 'bottom',
        reorder_cmap = [0, 4, 2, 1, 3, 5, 6],
        #crop_x = [620, 750],
        #crop_y = [800, 930],
        colorbar_shape = {'vertical_gaps': 0.15}
    )
    fig.savefig(f'/data/project/AI4Omic/MASLD/results/ST/cell2location/{sample}_{clust_col}'+'.pdf', dpi=300, format='pdf', bbox_inches='tight')

In [7]:
# Reloading the post-c2l results
adata_vis = sc.read_h5ad(f"{run_name}/adata_c2l.h5ad")


In [ ]:
adata_stamp = sc.read_h5ad('/data/project/AI4Omic/MASLD/results/ST/STAMP/adata_concat.h5ad')
topic_columns = [f'Topic{i}' for i in range(1, 21)]  # This will create Topic1 to Topic20
adata_vis.obs = pd.merge(adata_vis.obs, adata_stamp.obs[topic_columns], right_index=True, left_index=True, how='right')

In [25]:
adata_sub.obs.rename(columns={'Teff': 'Teff_CD8'})

,in_tissue,array_row,array_col,disease_status,slide,Age,Sex,Steatosis,Ballooning,Lobular_inflammation,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20
MASH-0422_AAACATTTCCCGGATT-1,1.0,61.0,97.0,MASH,MASH-0422,18.0,M,3.0,2.0,1.0,...,0.024318,0.017225,0.012946,0.027466,0.016620,0.012137,0.016501,0.015792,0.046853,0.363065
MASH-0422_AAACCTAAGCAGCCGG-1,1.0,65.0,83.0,MASH,MASH-0422,18.0,M,3.0,2.0,1.0,...,0.043299,0.064156,0.016984,0.046401,0.018418,0.019067,0.063840,0.060212,0.226153,0.127925
MASH-0422_AAACGGGCGTACGGGT-1,1.0,65.0,91.0,MASH,MASH-0422,18.0,M,3.0,2.0,1.0,...,0.013526,0.003183,0.038747,0.011065,0.006197,0.003479,0.003312,0.003137,0.017712,0.132310
MASH-0422_AAACGGTTGCGAACTG-1,1.0,67.0,59.0,MASH,MASH-0422,18.0,M,3.0,2.0,1.0,...,0.018042,0.086095,0.003214,0.087480,0.018143,0.086072,0.081824,0.082760,0.220929,0.043365
MASH-0422_AAACTCGGTTCGCAAT-1,1.0,66.0,70.0,MASH,MASH-0422,18.0,M,3.0,2.0,1.0,...,0.011719,0.012333,0.007718,0.129930,0.035732,0.018965,0.013661,0.012161,0.034523,0.020354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MASH-8684_TTGTGCAGCCACGTCA-1,1.0,60.0,74.0,MASH,MASH-8684,55.0,F,3.0,2.0,2.0,...,0.031779,0.095733,0.022650,0.033606,0.059226,0.052515,0.097122,0.098304,0.048313,0.066263
MASH-8684_TTGTGGTGGTACTAAG-1,1.0,63.0,95.0,MASH,MASH-8684,55.0,F,3.0,2.0,2.0,...,0.037553,0.075420,0.047972,0.052197,0.156103,0.073803,0.078776,0.078873,0.021354,0.010747
MASH-8684_TTGTTGTGTGTCAAGA-1,1.0,31.0,77.0,MASH,MASH-8684,55.0,F,3.0,2.0,2.0,...,0.044456,0.049761,0.036121,0.061595,0.167643,0.056359,0.054257,0.052263,0.019836,0.013967
MASH-8684_TTGTTTCACATCCAGG-1,1.0,58.0,42.0,MASH,MASH-8684,55.0,F,3.0,2.0,2.0,...,0.015541,0.005784,0.200082,0.019718,0.010346,0.003996,0.006180,0.005885,0.003388,0.014734


In [28]:
cell_types

Index(['Bmemory', 'Bnaive', 'CD14+ Monocyte', 'CD16+ Monocyte',
       'Central Vein EC', 'Cholangiocyte', 'Circulating NK cell',
       'Circulating NKT cell', 'HSC', 'Hepatocyte', 'KC', 'LAM', 'LSEC',
       'MAIT', 'Mast cell', 'Neutrophil', 'Plasma_IgA', 'Plasma_IgG',
       'Portal Vein EC', 'Resident NK cell', 'Teff_CD8', 'Tnaive/CD4/CD8',
       'Treg', 'cDC1', 'cDC2', 'pDC'],
      dtype='object')

In [ ]:
cell_types = inf_aver.columns
topics = topic_columns
# Create a correlation matrix between cell types and topics
import seaborn as sns
correlation_data = pd.DataFrame()

# Calculate correlations between cell types and topics
adata_sub = adata_vis[adata_vis.obs['disease_status'] == 'MASH']
adata_sub.obs = adata_sub.obs.rename(columns={'Teff': 'Teff_CD8', 'Tnaive': 'Tnaive_CD4/CD8'})
adata_sub.obs.rename(columns={'sample': 'slide'}, inplace=True)
for cell_type in cell_types:
    for topic in topics:
        corr = adata_sub.obs[cell_type].corr(adata_sub.obs[topic])
        correlation_data.loc[cell_type, topic] = corr

# Create heatmap
plt.figure(figsize=(10, 13))
cmap = sns.blend_palette(["#fafafa", '#dbc8db', '#d07aa4', '#926390', '#2d204c'], as_cmap=True)
sns.heatmap(correlation_data, cmap='RdBu_r', center=0, annot=True, fmt='.2f', cbar_kws={'label': 'Pearson Correlation'})
plt.title('')
plt.tight_layout()
plt.grid(False)
ax = plt.gca()
ax.collections[0].colorbar.remove()
plt.savefig(f'{results_folder}/celltype_topic_correlation.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [6]:
#Reload cell2location results
#mod = cell2location.models.Cell2location.load(f'{run_name}',adata_vis)
adata_vis = sc.read_h5ad(f"{run_name}/sp.h5ad")